# Processing the final dataset



In [1]:
import ee
import geemap

# Authenticate to Earth Engine
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

data_folder = "projects/amazon-forest-regrowth/assets"




Successfully saved authorization token.


In [31]:
# region = "countrywide"
# region = "amaz" #for Amazonia
region = "atl" #for Mata Atlantica

export_mature_biomass = False
export_distance = False

if region == "countrywide":
    # Countrywide
    region_shp = ee.FeatureCollection(f"{data_folder}/raw/biomes_br").geometry().dissolve()
elif region == "amaz":
    # Amazonia = 1
    region_shp = ee.FeatureCollection(f"{data_folder}/raw/biomes_br") \
        .filter(ee.Filter.eq("CD_Bioma", 1)).geometry()
elif region == "atl":
    region_shp = ee.FeatureCollection(f"{data_folder}/raw/biomes_br") \
        .filter(ee.Filter.eq("CD_Bioma", 3)).geometry()


In [7]:
mature_biomass1 = ee.Image(f"{data_folder}/nearest_mat_biomass_1000m_countrywide").select('mature_biomass')#.clip(region_shp)
mature_biomass2 = ee.Image(f"{data_folder}/nearest_mat_biomass_500m_amaz").select('b2')#.clip(region_shp)
mature_biomass = ee.Image(f"{data_folder}/mature_biomass_100m_countrywide")
distance = ee.Image(f"{data_folder}/distance_1000m_countrywide")
mature_biomass_500m_countrywide = ee.Image(f"{data_folder}/mature_biomass_500m_countrywide")

map = geemap.Map(center=(-12, -55), zoom=4)
map.addLayer(mature_biomass, {'min':0, 'max': 400, 'palette':['blue', 'red']}, "mature_biomass")
map.addLayer(mature_biomass1, {'min':0, 'max': 400, 'palette':['blue', 'red']}, "mature_biomass1")
map.addLayer(mature_biomass2, {'min':0, 'max': 400, 'palette':['blue', 'red']}, "mature_biomass2")
map.addLayer(distance, {'min':0, 'max': 40000, 'palette':['blue', 'red']}, "distance")
map.addLayer(mature_biomass_500m_countrywide, {'min':0, 'max': 400, 'palette':['blue', 'red']}, "mature_biomass_500m_countrywide")
map

Map(center=[-12, -55], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(ch…

In [40]:
def export_clipped_image(image, scale, export_type, image_name=None):
    """
    Clips and exports image to Google Earth Engine asset and/or Google Drive.

    Parameters:
    - image: ee.Image, the image to export.
    - scale: int, the scale in meters.
    - region: str, 'amaz' or 'all_country'.
    - export_type: str, 'asset', 'drive', or 'both'.
    """
    
    # Prepare export parameters
    export_params = {
        'image': image.clip(region_shp),
        'scale': scale,
        'region': region_shp,
        'crs': "EPSG:4326",
        'maxPixels': 1e13
    }
    
    # Set additional parameters for each export type
    if export_type in ['asset', 'both']:
        asset_params = export_params.copy()
        asset_params['assetId'] = f"projects/amazon-forest-regrowth/assets/{image_name}_{scale}m_{region}"
        asset_params['description'] = f"{image_name}_{scale}m_{region}_toAsset"
        asset_task = ee.batch.Export.image.toAsset(**asset_params)
        asset_task.start()
        print(f"Asset export task for {image_name} in {region} started.")
    
    if export_type in ['drive', 'both']:
        drive_params = export_params.copy()
        drive_params['fileNamePrefix'] = f'{image_name}_{scale}m_{region}'
        drive_params['description'] = f"{image_name}_{scale}m_{region}_toDrive"
        drive_task = ee.batch.Export.image.toDrive(**drive_params)
        drive_task.start()
        print(f"Drive export task for {image_name} in {region} started.")



In [42]:
mature_biomass = ee.Image(f"{data_folder}/mature_biomass_30m_countrywide").clip(region_shp)

if export_mature_biomass:
    export_clipped_image(mature_biomass, 500, 'both', f"mature_biomass_500m_{region}")

In [18]:
mature_biomass_aggregated = ee.Image(f"{data_folder}/mature_biomass_500m_{region}")

age_agbd = ee.Image(f"{data_folder}/age_agbd")

distance = mature_biomass_aggregated.fastDistanceTransform().sqrt() \
    .multiply(ee.Image.pixelArea().sqrt()).rename("distance") \
    .updateMask(age_agbd.select("age")).float()

if export_distance:
    export_clipped_image(distance, 1000, 'both', 'distance')

In [30]:
distance = ee.Image(f"{data_folder}/distance_1000m_{region}")

export_task = ee.batch.Export.image.toDrive(
    image=distance,
    description='distance_1000m_countrywide',
    region = region_shp,
    scale=1000,
    maxPixels=1e13
)
export_task.start()

In [25]:
age_agbd = ee.Image(f"{data_folder}/age_agbd")
distance = ee.Image(f"{data_folder}/distance_1000m_{region}")

# Sample the image to extract all points
points = age_agbd.sample(
    region=region_shp,
    scale=30,  # Adjust the scale as needed
    numPixels=1e13,  # Large number to ensure all points are sampled
    geometries=True  # Include geometries in the output
)

pts_dist = distance.reduceRegions(points, ee.Reducer.first().setOutputs(["distance"]), scale = 1000)

# pts_dist = pts_dist.map(lambda feature: ee.Feature(None, {'distance': feature.get('distance')}))

# Function to buffer and calculate mean
def buffer_and_calculate_mean(f):
    distance = ee.Number(f.get('distance'))
    buffered = ee.Feature(f.buffer(distance.add(1000), 1))
    mean_value = mature_biomass_aggregated.reduceRegion(ee.Reducer.mean().unweighted(), \
                                                        buffered.geometry(), 1000, maxPixels=1e13)
    return buffered.set(mean_value)

pts = pts_dist.map(buffer_and_calculate_mean)

pts_image = pts.reduceToImage(properties=['distance'], reducer=ee.Reducer.first())

export_task = ee.batch.Export.image.toAsset(
    image=pts_image,
    assetId=f"projects/amazon-forest-regrowth/assets/nearest_distance_1000m_countrywide_gee",
    description='nearest_distance_1000m_countrywide_gee',
    region = region_shp,
    scale=1000,
    maxPixels=1e13
)
export_task.start()


In [7]:
def export_csv(year):
    if (year == 0):
        land_use_years = ee.Image(f"{data_folder}/land_use")
        name = f"{region}_all"
    else:
        land_use_years = ee.Image(f"{data_folder}/land_use_{year}_years")
        name = f"{region}_{year}yr"

    age_agbd = ee.Image(f"{data_folder}/age_agbd")
    categorical = ee.Image(f"{data_folder}/categorical")
    yearly_mean_prec = ee.Image(f"{data_folder}/yearly_mean_prec")
    yearly_SI = ee.Image(f"{data_folder}/yearly_SI")
    cwd = ee.Image(f"{data_folder}/raw/cwd_chave")
    sur_cover = ee.Image(f"{data_folder}/sur_cover").rename("sur_cover")
    one_hectare_mask = ee.Image(f"{data_folder}/one_hectare_mask")
    mat = ee.Image(f"{data_folder}/nearest_mat_biomass_1000m_{region}").rename("distance", "mature_biomass")
    
    combined_mask = one_hectare_mask.And(land_use_years.select("last_LU")) \
        .And(mat.select("distance"))

    unified_data = (
        age_agbd.addBands(
            [mat, categorical, land_use_years, cwd, sur_cover, yearly_mean_prec, yearly_SI
        ]).updateMask(combined_mask)
    )

    unified_data_fc = unified_data.stratifiedSample(
        numPoints = 5000, classBand = 'ecoreg', region = unified_data.geometry()
    )

    # Export an ee.FeatureCollection to Google Drive, adjusting the description based on the years
    task = ee.batch.Export.table.toDrive(
        collection=unified_data_fc, description=name
    )
    task.start()

export_csv(0)
export_csv(5)
export_csv(10)
export_csv(15)

In [16]:
ecoregion = ee.Image(f"{data_folder}/categorical").select("ecoreg")
biome_ecoreg = ee.Image(f"{data_folder}/categorical").select("biome")
biome_ibge = ee.FeatureCollection(f"{data_folder}/raw/biomes_br")
age_agbd = ee.Image(f"{data_folder}/age_agbd")

map = geemap.Map()
map.addLayer(ecoregion, {'min': 400, 'max':620, 'palette':['red', 'blue']}, "ecoregion")
map.addLayer(biome_ecoreg, {'min': 1, 'max':6, 'palette':['red', 'blue']}, "biome_ecoreg")
map.addLayer(biome_ibge, {}, "biomes")
map.addLayer(age_agbd.select("age"), {}, "age_agbd")
map


Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…